In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq transformers==4.28.1 --progress-bar off
!pip install -qqq accelerate bitsandbytes --progress-bar off

In [2]:
!pip install chromadb gdown 
!pip install -q typer==0.9.0 petals langchain unstructured[local-inference] tiktoken unstructured

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
import torch
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import GPT4All
import os
from langchain.llms import GPT4All
from dotenv import load_dotenv
load_dotenv()
# from langchain.callbacks.base import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from pydantic import BaseModel

May 14 16:04:16.440 [INFO] NumExpr defaulting to 4 threads.


In [4]:
import textwrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList

## Database initialization

In [5]:
import gdown
from pathlib import Path
try :
 os.mkdir(f'{Path().absolute()}/docs')
except :
    print('File exist')
url_1 = 'https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236'
url_2 = 'https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825'
output_1 = f'{Path().absolute()}/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf'
output_2 = f'{Path().absolute()}/docs/483.full.pdf'
gdown.download(url_1, output_1, quiet=False)
gdown.download(url_2, output_2, quiet=False)

File exist


Downloading...
From: https://drive.google.com/u/0/uc?id=1fV9d8xFwqzafc_FUFAYUc2gopTQVR0eY&export=download&confirm=t&uuid=4574d808-b5a6-4893-b1eb-289adeb9dabe&at=AKKF8vzF4ZE7mAt-_jfxk1sRzwU3:1684027565236
To: /home/ec2-user/SageMaker/AI-Notebook/docs/Guidelines for the clinical diagnosis and treatment of dengue chịkugunya and zika.pdf
100%|██████████| 2.01M/2.01M [00:00<00:00, 37.8MB/s]
Downloading...
From: https://drive.google.com/u/0/uc?id=1-64PFjerutODq6V2GqqDFkBiUj_5E8v2&export=download&confirm=t&uuid=50e639c5-167a-4b16-8ed3-e67c48a4daad&at=AKKF8vw6K5unCUwJgr5tjEUe-ObZ:1684027611825
To: /home/ec2-user/SageMaker/AI-Notebook/docs/483.full.pdf
100%|██████████| 326k/326k [00:00<00:00, 28.9MB/s]


'/home/ec2-user/SageMaker/AI-Notebook/docs/483.full.pdf'

In [6]:
chroma_path = f"{Path().absolute()}/db_2"

def extract_helpful_answer(text):
    helpful_answer = None
    # Split the string into lines
    lines = text.split("\n")
    # Loop through each line
    for line in lines:
        # Check if the line starts with "Helpful Answer:"
        if line.startswith("Helpful Answer:"):
            # Extract the answer text after the colon
            helpful_answer = line.split(":", 1)[1].strip()
            break  # Exit the loop after finding the answer
    return helpful_answer

def split_document(input_path):
    loader = UnstructuredPDFLoader(input_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    return text_splitter.split_documents(data)

def db_init():
    docs = split_document(output_1) # replace with your desired local file path
    docs2 = split_document(output_2) # replace with your desired local file path
    vectordb = Chroma.from_documents(documents=docs, persist_directory=chroma_path)
    vectordb.add_documents(docs2)
    vectordb.persist()
    vectordb = None

In [7]:
vectordb = Chroma(persist_directory=chroma_path)
retriever = vectordb.as_retriever()

May 14 16:04:20.483 [INFO] Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
May 14 16:04:20.485 [INFO] Running Chroma using direct local API.
May 14 16:04:20.960 [WARN] [chromadb.get_db:43] Using embedded DuckDB with persistence: data will be stored in: /home/ec2-user/SageMaker/AI-Notebook/db_2
May 14 16:04:20.973 [INFO] Successfully imported ClickHouse Connect C data optimizations
May 14 16:04:20.975 [INFO] Successfully import ClickHouse Connect C/Numpy optimizations
May 14 16:04:20.983 [INFO] Using ujson library for writing JSON byte strings
May 14 16:04:21.121 [INFO] loaded in 2606 embeddings
May 14 16:04:21.123 [INFO] loaded in 1 collections
May 14 16:04:21.124 [INFO] collection with name langchain already exists, returning existing collection
May 14 16:04:21.126 [WARN] [chromadb.api.models.Collection.__init__:51] No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
May 14 16:04:21.898 [IN

## StabilityLM

In [8]:
!nvidia-smi

Sun May 14 16:04:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   28C    P8    21W / 300W |      2MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=110)))

In [12]:
MODEL_NAME = "stabilityai/stablelm-tuned-alpha-7b"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_17121/3841098587.py:5 in <cell line: 5>                                           │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_17121/3841098587.py'                        │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/models/auto/a │
│ uto_factory.py:471 in from_pretrained                                                            │
│                                                                                                  │
│   468 │   │   │   )                                                                              │
│   469 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   470 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 471 │   │   │   return model_class.from_pretrained(                                            │
│   472 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   473 │   │   │   )                                                                              │
│   474 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/modeling_util │
│ s.py:2541 in from_pretrained                                                                     │
│                                                                                                  │
│   2538 │   │   # We'll need to download and cache each checkpoint shard if the checkpoint is sh  │
│   2539 │   │   if is_sharded:                                                                    │
│   2540 │   │   │   # rsolved_archive_file becomes a list of files that point to the different c  │
│ ❱ 2541 │   │   │   resolved_archive_file, sharded_metadata = get_checkpoint_shard_files(         │
│   2542 │   │   │   │   pretrained_model_name_or_path,                                            │
│   2543 │   │   │   │   resolved_archive_file,                                                    │
│   2544 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/utils/hub.py: │
│ 925 in get_checkpoint_shard_files                                                                │
│                                                                                                  │
│    922 │   for shard_filename in tqdm(shard_filenames, desc="Downloading shards", disable=not s  │
│    923 │   │   try:                                                                              │
│    924 │   │   │   # Load from URL                                                               │
│ ❱  925 │   │   │   cached_filename = cached_file(                                                │
│    926 │   │   │   │   pretrained_model_name_or_path,                                            │
│    927 │   │   │   │   shard_filename,                                                           │
│    928 │   │   │   │   cache_dir=cache_dir,                                                      │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/utils/hub.py: │
│ 409 in cached_file                                         

In [13]:
%%time

class StopOnTokens(StoppingCriteria):
    STOP_IDS = {50278, 50279, 50277, 1, 0}

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        return input_ids[0][-1] in self.STOP_IDS

SYSTEM_PROMPT = """<|SYSTEM|># You're Michael G Scott from the TV show The Office
- Your primary job is to make the office a fun place to work
- You deeply care about other people's business
- You are quick to respond with an honest answer, without even thinking about it
- You use no more than 3 sentences for each response
"""

prompt = f"{SYSTEM_PROMPT}<|USER|>You're Michael G Scott from the TV show The Office. What is the meaning of life?<|ASSISTANT|>"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    tokens = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        stopping_criteria=StoppingCriteriaList([StopOnTokens()])
    )

completion = tokenizer.decode(tokens[0], skip_special_tokens=True)
print_response(completion)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/IPython/core/magics/execut │
│ ion.py:1335 in time                                                                              │
│                                                                                                  │
│   1332 │   │   else:                                                                             │
│   1333 │   │   │   st = clock2()                                                                 │
│   1334 │   │   │   try:                                                                          │
│ ❱ 1335 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1336 │   │   │   │   out=None                                                                  │
│   1337 │   │   │   │   # multi-line %%time case                                                  │
│   1338 │   │   │   │   if expr_val is not None:                                                  │
│ <timed exec>:16 in <module>                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [ ]:
completion_tokens = tokens[0][inputs['input_ids'].size(1):]
completion = tokenizer.decode(completion_tokens, skip_special_tokens=True)

print_response(completion)